# Assignment

here comes info about the assignment

## Import libraries

here come all the imported libraries

In [32]:
import pandas as pd
from pathlib import Path
import numpy as np
import math
import scipy
from scipy.signal import find_peaks
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import geopandas as gpd


## Part I - Data import

Here comes the first part of the assignment. Importing the filtered data. Create a dataframe.
Show the data in a table.


In [33]:
#file_path = 'Greenhouse_gas_emissions.csv'
#file_path = 'NOx_data.csv'
#file_path = '2.5.csv'
file_path = '10.csv'


df = pd.read_csv(file_path, sep = ';',encoding='ISO-8859-1') 

# The file had an incorrect delimiter which caused a ParserError,
# this is solved by adding sep=';' when reading the file. 

In [34]:
df

,PM10,2019,2020
0,Belgium,"27,438","25,513"
1,Bulgaria,"44,091","44,525"
2,Czechia,"40,567","37,402"
3,Denmark,"23,350","22,412"
4,Germany,"195,887","182,126"
5,Estonia,"11,780","12,664"
6,Ireland,"30,461","30,287"
7,Greece,"58,434","56,342"
8,Spain,"211,498","211,698"
9,France,"277,876","247,518"


In [35]:
# Maybe we should merge the data so that we have one row per country (instead of now per city within a country)
# We need to filter the data, get rid of columns that we will not look at.

## Part II - Data processing

Peak finding. Check of common peaks between data sets. 

## Part III - Data visualisation

Conclusion. Relate found info from Part II to our RQ. Perhaps with graphs, maps, etc. 